In [1]:
import os
import pandas as pd
import numpy as np


d = {
    'carpet': 'Ковёр',
    'grid': 'Металлическая сетка',
    'leather': 'Кожа',
    'tile': 'Клетка',
    'wood': 'Дерево',
    'bottle': 'Бутылка',
    'cable': 'Кабель',
    'capsule': 'Капсула',
    'hazelnut': 'Орех',
    'metal_nut': 'Гайка',
    'pill': 'Таблетка',
    'screw': 'Винт',
    'toothbrush': 'Зубная счётка',
    'transistor': 'Транзистор',
    'zipper': 'Застёжка-молния'
}

light_archs = [
    'squeezenet1_0',
    'squeezenet1_1',
    'shufflenet_v2_x0_5',
    'shufflenet_v2_x1_0',
    'mobilenet_v2',
    'mobilenet_v3_small',
    'mobilenet_v3_large',
    'mnasnet0_5',
    'mnasnet1_0'
]
archs = ['shufflenet_v2_x0_5', 'shufflenet_v2_x1_0']

In [12]:
s = '''
alexnet 0.72 61.1
vgg11 7.63 132.86
vgg13 11.34 133.05
vgg16 15.5 138.36
vgg19 19.67 143.67
vgg11_bn 7.64 132.87
vgg13_bn 11.36 133.05
vgg16_bn 15.53 138.37
vgg19_bn 19.7 143.68
resnet18 1.82 11.69
resnet34 3.68 21.8
resnet50 4.12 25.56
resnet101 7.85 44.55
resnet152 11.58 60.19
squeezenet1_0 0.83 1.25
squeezenet1_1 0.36 1.24
densenet121 2.88 7.98
densenet169 3.42 14.15
densenet201 4.37 20.01
densenet161 7.82 28.68
inception_v3 2.85 27.16
efficientnet_b0 0.39 5.3
efficientnet_b1 0.7 7.8
efficientnet_b2 1.0 9.2
efficientnet_b3 1.8 12
efficientnet_b4 4.2 19
efficientnet_b5 9.9 30
efficientnet_b6 19.0 43
efficientnet_b7 37.0 66
regnet_x_16gf 16 54.3
regnet_x_1_6gf 1.6 9.2
regnet_x_32gf 32 107.8
regnet_x_3_2gf 3.2 15.3
regnet_x_400mf 0.4 5.2
regnet_x_800mf 0.8 7.3
regnet_x_8gf 8 39.6
regnet_y_16gf 16 83.6
regnet_y_1_6gf 1.6 11.2
regnet_y_32gf 32 145
regnet_y_3_2gf 3.2 19.4
regnet_y_400mf 0.4 4.3
regnet_y_800mf 0.8 6.3
regnet_y_8gf 8 39.2
convnext_base 15.4 89
convnext_large 34.4 198
convnext_small 8.7 50
convnext_tiny 4.5 29
mnasnet0_5 0.31 3.9
mnasnet1_0 0.34 4.8
mobilenet_v2 0.3 3.4
mobilenet_v3_large 0.22 5.4
mobilenet_v3_small 0.06 2.5
googlenet 1.5 11
shufflenet_v2_x0_5 0.04 1.4
shufflenet_v2_x1_0 0.15 2.3
resnext50_32x4d 4.1 25
resnext101_32x8d 7.8 44
wide_resnet101_2 22.7 126.9
wide_resnet50_2 11.4 68.9
resnest101 17 48
resnest50 5.4 27.5
'''

len(list(map(lambda c: c.split(), [c for c in s.split('\n') if c != ''])))

61

In [7]:
all_archs = dict()

for f in [file for file in os.listdir() if os.path.isfile(file)]:
    if f[-3:] == 'csv':
        all_archs[f[7:][:-4]] = []

In [76]:
print("""
\\hline
& \\multicolumn{2}{c|}{PaDiM} \\\\ \\hline
Бэкбон & ROC-AUC & pROC-AUC & Время работы \\\\ \\hline"""
)

for arch in archs:
    res = pd.read_csv(f'result_{arch}.csv').loc[15]
    
    i = int(res['Image'] * 1000) / 10
    p = int(res['Pixel'] * 1000) / 10
    t = round(res['Time'], 3)
    
    print(f'{arch} & {i}\% & {p}\% & {t} c. \\\\')
    
print('\\hline')


\hline
& \multicolumn{2}{c|}{PaDiM} \\ \hline
Бэкбон & ROC-AUC & pROC-AUC & Время работы \\ \hline
shufflenet_v2_x0_5 & 92.6\% & 95.8\% & 0.013 c. \\
shufflenet_v2_x1_0 & 94.3\% & 96.3\% & 0.006 c. \\
\hline


In [55]:
df_full = pd.DataFrame(columns=['Type', 'Final'])

for arch in archs:
    df = pd.read_csv(f'result_{arch}.csv').loc[:14]
    
    df_new = pd.DataFrame(columns=df.columns)

    for key in d.keys():
        df_new = pd.concat((df_new, df[df['Type'] == key]))

    df_new = df_new.reset_index()[df.columns]

    df_new_texture = df_new.loc[df_new['Type'].isin(['carpet', 'grid', 'leather', 'tile', 'wood'])]
    df_new_obj = df_new.loc[~df_new['Type'].isin(['carpet', 'grid', 'leather', 'tile', 'wood'])]

    df_new['Type'] = df_new['Type'].apply(lambda x: d[x])

    df_new = pd.concat((
        df_new,
        pd.DataFrame(
            [[
                'Текстурные классы',
                df_new_texture['Image'].mean(),
                df_new_texture['Pixel'].mean(),
                df_new_texture['Time'].mean()
            ]],
            columns=df.columns
        ),
        pd.DataFrame(
            [[
                'Объектные классы',
                df_new_obj['Image'].mean(),
                df_new_obj['Pixel'].mean(),
                df_new_obj['Time'].mean()
            ]],
            columns=df.columns
        ),
        pd.DataFrame(
            [[
                'Все классы',
                df_new['Image'].mean(),
                df_new['Pixel'].mean(),
                df_new['Time'].mean()
            ]],
            columns=df.columns
        )
    ))

    df_new['Image'] = (df_new['Image'] * 1000).astype(np.int64) / 10
    df_new['Pixel'] = (df_new['Pixel'] * 1000).astype(np.int64) / 10
    
    df_new['Record'] = '(' + df_new['Image'].astype(str) \
                           + '\%, ' + df_new['Pixel'].astype(str) + '\%, '\
                           + df_new['Time'].astype(str).apply(lambda x: x[:5]) + ')'

    df_new = df_new[['Type', 'Record']]
    
    
    if len(df_full) == 0:
        df_full = df_new.copy()
        df_full.columns = ['Type', 'Final']
    else:
        df_full = pd.concat((df_full, df_new[['Record']]), axis=1)
        df_full['Final'] = df_full['Final'] + ' & ' + df_full['Record']
        df_full = df_full[['Type', 'Final']]

In [56]:
df_new

,Type,Record
0,Ковёр,"(93.7\%, 97.2\%, 0.0)"
1,Металлическая сетка,"(91.9\%, 92.6\%, 0.0)"
2,Кожа,"(99.9\%, 97.9\%, 0.0)"
3,Клетка,"(98.0\%, 91.9\%, 0.014)"
4,Дерево,"(99.8\%, 91.4\%, 0.014)"
5,Бутылка,"(100.0\%, 97.6\%, 0.0)"
6,Кабель,"(94.6\%, 96.6\%, 0.014)"
7,Капсула,"(87.8\%, 98.0\%, 0.0)"
8,Орех,"(94.8\%, 98.1\%, 0.014)"
9,Гайка,"(97.9\%, 97.6\%, 0.0)"


In [6]:
result = df_full['Type'] + ' & ' + df_full['Final'] + ' \\\\'

for record in result:
    print(record)

Ковёр & (91.3\%, 96.7\%) & (93.7\%, 97.2\%) \\
Металлическая сетка & (87.8\%, 89.8\%) & (91.9\%, 92.6\%) \\
Кожа & (99.7\%, 97.8\%) & (99.9\%, 97.9\%) \\
Клетка & (97.8\%, 90.7\%) & (98.0\%, 91.9\%) \\
Дерево & (100.0\%, 92.2\%) & (99.8\%, 91.4\%) \\
Бутылка & (99.6\%, 96.9\%) & (100.0\%, 97.6\%) \\
Кабель & (95.7\%, 96.5\%) & (94.6\%, 96.6\%) \\
Капсула & (85.6\%, 97.9\%) & (87.8\%, 98.0\%) \\
Орех & (94.0\%, 98.1\%) & (94.8\%, 98.1\%) \\
Гайка & (99.3\%, 96.9\%) & (97.9\%, 97.6\%) \\
Таблетка & (80.5\%, 95.2\%) & (88.7\%, 96.0\%) \\
Винт & (73.8\%, 95.5\%) & (82.3\%, 96.6\%) \\
Зубная счётка & (100.0\%, 98.2\%) & (98.3\%, 98.4\%) \\
Транзистор & (97.5\%, 97.9\%) & (98.5\%, 98.1\%) \\
Застёжка-молния & (86.5\%, 96.3\%) & (88.3\%, 96.8\%) \\
Текстурные классы & (95.3\%, 93.4\%) & (96.7\%, 94.2\%) \\
Объектные классы & (95.3\%, 93.4\%) & (96.7\%, 94.2\%) \\
Все классы & (92.6\%, 95.8\%) & (94.3\%, 96.3\%) \\
